In [3]:
import random
import numpy as np
import math
import pandas as pd

In [4]:
with open('rowZip.csv', 'r') as file:
    rows = file.readlines()
rows = [[int(st) for st in row.split(',')] for row in rows]
rows[0]

[11, 42, 71, 108, 119, 174, 215, 246, 284, 461, 540, 547, 641, 755, 756]

In [5]:
with open('colZip.csv', 'r') as file:
    cols = file.readlines()
cols = [[int(st) for st in col.split(',')] for col in cols]
cols[0]

[0, 1, 2, 3, 5, 6, 7, 11, 12, 16, 20, 26, 27, 37, 47, 48, 50, 113, 125, 128]

def user_ranstr(num):
    userID = ''
    H = 'AB'
    userID += random.choice(H)
    H = '0123456789'
    for i in range(num-1):
        userID += random.choice(H)
    return userID


def item_ranstr(num):
    itemID = ''
    H = 'def'
    itemID += random.choice(H)
    H = '0123456789'
    for i in range(num-1):
        itemID += random.choice(H)
    return itemID

In [6]:
def initial_dataset(n,num,num2):
    #n: grid width, num: length of user/item ID , num2: max_bicluster_width
    bicluster = []
    for i in range(n*n):
        '''num_user = random.randrange(num2)
        num_item = random.randrange(num2)
        userID = []
        itemID = []
        j = 0
        k = 0
        while j < num_user:
            try:
                user = user_ranstr(num)
                userID.index(user)
            except:
                userID.append(user)
                j= j + 1 
        while k < num_item:
            try:
                item = item_ranstr(num)
                itemID.index(item)
            except:
                itemID.append(item)
                k = k + 1'''
        userID = rows[i]
        itemID = cols[i]
        single_set = [userID,itemID]
        bicluster.append(single_set)
    return bicluster

In [7]:
def reward_func(set1,set2):   
    intersection = 0
    for i in range(len(set2)):
        try:
            set1.index(set2[i])
            intersection = intersection + 1
     #       print(set2[i])
        except:
            pass 
   ## print("交:",intersection)
   # print("並:",(len(set1)+len(set2)-intersection))
    try:
        reward = intersection/(len(set1)+len(set2)-intersection)
    except:
        reward = 1
    return round(reward,3)

In [8]:
def gen_gridworld(width, data):
    left_num = width*width
    gridworld = [[0 for col in range(width)] for row in range(width)]
    for i in range(width):
        for j in range(width):
            num = random.randrange(left_num)
            gridworld[i][j] = data[num]
            data.remove( data[num])
            left_num = left_num - 1
    return gridworld

In [9]:
def gen_reward_grid(gridworld,width):
    grid_reward = [[0 for col in range(width)] for row in range(width)]
    for i in range(width):
        for j in range(width):
            reward=[]
            #print("i=",i,"j=",j)
            if j%width == width-1:
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)  
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)       
            elif j%width == 0:
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))    
            else: 
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))   
            grid_reward[i][j] = reward
    return grid_reward

In [10]:
def total_reward(grid_reward):
    return np.sum(grid_reward)

In [11]:
def hilclimb(tot_reward,reward_grid,data_grid,width,alpha, beta,iteration):
    i = random.randrange(width)
    j = random.randrange(width)
    i2 = random.randrange(width)
    j2 = random.randrange(width)
    data_grid[i][j], data_grid[i2][j2] =  data_grid[i2][j2], data_grid[i][j]
    reward_update(reward_grid,data_grid,i,j, width)
    reward_update(reward_grid,data_grid,i2,j2, width)
    new_tot_reward = total_reward(reward_grid)
    P_k = 1.0-beta*alpha**iteration
    #print("P_k:",P_k)
    backfire = random.random()
   # print("backfire:",backfire)
    T = 10000000
    flag = (iteration % T == 0)
    if P_k < backfire:
        if flag:
            print("backfire,old:",tot_reward," new:",new_tot_reward)
        return new_tot_reward, 1
    else: 
        if new_tot_reward > tot_reward:
            if flag:
                print("update:",new_tot_reward)
            return new_tot_reward, 1
        else:
            if flag:
                print("non_update:",tot_reward," new:F",new_tot_reward)
            data_grid[i][j],  data_grid[i2][j2] =   data_grid[i2][j2],  data_grid[i][j]
            reward_update(reward_grid,data_grid,i,j, width)
            reward_update(reward_grid,data_grid,i2,j2, width)
            return tot_reward, 0

In [12]:
def reward_update(grid_reward,gridworld,x,y, width):
    x_update = [x-1,x,x+1,x,x]
    y_update = [y,y,y,y-1,y+1]
    for i, j in zip(x_update, y_update):
        reward=[]
        #print("i=",i,"j=",j)
        if i < 0 or i > width-1 or j < 0 or j> width-1:
           # print("i=",i,"j=",j)
            continue
        if j%width == width-1:
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)  
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)       
        elif j%width == 0:
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))    
        else: 
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))   
        grid_reward[i][j] = reward

In [13]:
width = 20
id_length = 2
max_bicluster_width = 20
beta = 0.4
stop_value = 10000

data = initial_dataset(width,id_length,max_bicluster_width)
data_grid = gen_gridworld(width, data)
reward_grid = gen_reward_grid(data_grid,width)
tot_reward = total_reward(reward_grid)
print(tot_reward)
stop = 0
iteration = 0
alpha = 0.99999
while stop < stop_value:
    iteration = iteration + 1
    tot_reward, update_bool = hilclimb(tot_reward,reward_grid,data_grid,width,alpha, beta, iteration)
    P_k = 1.0-beta*alpha**iteration
    if update_bool == 1:
        stop = 0
    else:
        stop = stop+1
    if iteration % 10000 == 0:
        print(P_k, tot_reward)
    if 1 - P_k < 1e-5:
        break
    #print("real: ", total_reward(gen_reward_grid(data_grid,width)))
print(tot_reward)

774.358
0.6380652137540963 812.266
0.67250802626283 804.182
0.7036731562204706 810.212
0.7318725178443317 807.0519999999999
0.7573883426483397 815.7739999999999
0.780476004139156 838.012
0.8013665737056469 837.674
0.8202691332821546 845.72
0.8373728679317141 844.97
0.8528489592927013 838.262
0.866852298839354 859.338
0.8795230381032203 847.822
0.8909879913708328 879.3739999999999
0.9013619048964144 856.444
0.9107486053324516 876.9559999999999
0.9192420388721302 884.9879999999999
0.9269272115038236 891.324
0.9338810397881107 918.578
0.9401731206722711 897.962
0.9458664280468974 897.058
0.9510179430160663 908.568
0.9556792241890882 912.4179999999999
0.9598969237015627 914.9159999999999
0.9637132541303047 923.656
0.967166410975231 957.156
0.970290954936586 947.4159999999999
0.9731181578135083 948.2379999999998
0.97567631548584 944.8039999999999
0.9779910311116368 955.538
0.9800854713747438 970.7359999999999
0.981980598347075 973.4359999999999
0.9836953792861737 977.318
0.9852469764678018 

In [14]:
total_reward(gen_reward_grid(data_grid,width))

1227.772

In [15]:
data_grid[0][1]

[[42, 69, 71, 79, 119, 170, 174, 186, 216, 320, 461, 493, 518, 547, 571],
 [0, 1, 2, 3, 4, 5, 6, 8, 11, 14, 15, 26, 37, 39, 47, 49, 51, 78, 136, 147]]

In [16]:
import pickle

f1 = open("dic", "rb")
f2 = open("dic_inv", "rb")
dic = pickle.load(f1)
dic_inv = pickle.load(f2)
f1.close()
f2.close()

In [17]:
real_grid = [[[data_grid[k][j][0], [dic_inv[i] for i in data_grid[j][k][1]]] for j in range(width)] for k in range(width)]

In [18]:
f = open("grid", "wb")
pickle.dump(real_grid, f)
f.close()

In [19]:
f = open("grid", "rb")
pickle.load(f) == real_grid

True